In [ ]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

(X_train, y_train), (X_test, y_test)= mnist.load_data()

# Normalize pixel values
X_train = X_train/255.0
X_test = X_test/255.0

# Reshape data to include channel dimension (for grayscale, it's 1)
X_train = X_train.reshape(-1, 28, 28, 1)
X_test = X_test.reshape(-1, 28, 28, 1)

# Convert labels to one-hot encoding, which is required for 'categorical_crossentropy'
y_train = to_categorical(y_train, num_classes=10)
y_test = to_categorical(y_test, num_classes=10)


# Define the model with the correct input shape for MNIST
model = tf.keras.models.Sequential([
    tf.keras.layers.InputLayer(input_shape=(28, 28, 1)), # Changed input shape to match MNIST
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


Epoch 1/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 60s 38ms/step - accuracy: 0.8506 - loss: 0.4638 - val_accuracy: 0.9743 - val_loss: 0.0834
Epoch 2/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 77s 35ms/step - accuracy: 0.9766 - loss: 0.0774 - val_accuracy: 0.9784 - val_loss: 0.0703
Epoch 3/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 54s 36ms/step - accuracy: 0.9855 - loss: 0.0501 - val_accuracy: 0.9829 - val_loss: 0.0551
Epoch 4/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 83s 37ms/step - accuracy: 0.9882 - loss: 0.0365 - val_accuracy: 0.9848 - val_loss: 0.0465
Epoch 5/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 81s 36ms/step - accuracy: 0.9908 - loss: 0.0277 - val_accuracy: 0.9860 - val_loss: 0.0445
Epoch 6/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 81s 36ms/step - accuracy: 0.9933 - loss: 0.0211 - val_accuracy: 0.9855 - val_loss: 0.0509
Epoch 7/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 101s 48ms/step - accuracy: 0.9940 - loss: 0.0179 - val_accuracy: 0.9864 - val_loss: 0.0501
Epoch 8/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 70s 47ms/step - accuracy: 0.9955 

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with open('mnist_model.tflite', 'wb') as f:
    f.write(tflite_model)

converter.optimizations = [tf.lite.Optimize.DEFAULT]

Saved artifact at '/tmp/tmpcjuww2a0'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 28, 28, 1), dtype=tf.float32, name='keras_tensor_30')
Output Type:
  TensorSpec(shape=(None, 10), dtype=tf.float32, name=None)
Captures:
  139532448796880: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139532448798800: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139532448798224: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139532448799568: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139532448798992: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139532448800336: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139532448799760: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139532448801104: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139532448800528: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139532448801872: TensorSpec(shape=(), dtype=tf.resource, name=None)


In [ ]:
interpreter = tf.lite.Interpreter(model_path='mnist_model.tflite')
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
input_shape = input_details[0]['shape']

In [ ]:
input_shape

array([ 1, 28, 28,  1], dtype=int32)